In [ ]:
install.packages('mixtools')
install.packages('concordance')

In [8]:
library(readxl)
library(purrr)
library(dplyr)
library(tidyr)
library(haven)
library(mixtools)
library(concordance)
library(ggplot2)
library(stringr)
source('./explore.R')

In [129]:
dat.03 <- read_excel('./years/oesm03in4/nat4d_may2003_dl.xls')

In [130]:
dat.07 <- read_excel('./years//oesm07in4//nat4d_May2007_dl.xls')

In [131]:
dat.12 <- rbind(
    read_excel('./years/oesm12in4//oesm12in4//nat4d_M2012_dl_1_113300_517100.xls'), 
    read_excel('./years/oesm12in4//oesm12in4//nat4d_M2012_dl_2_517200_999300.xls'))
# dat.12 <- read_excel('./years/oesm12in4/oesm12in4//nat3d_M2012_dl.xls')

In [132]:
dat.16 <- read_excel('./years/oesm16in4/oesm16in4/nat4d_M2016_dl.xlsx')

In [5]:
codes <- dat.03 %>% group_by(NAICS) %>% count() %>% arrange(n) %>% tail(10) %>% pull(NAICS)

In [6]:
extract.wb <- function(path) {
    v <- read.csv(path, skip=3) %>% filter(Country.Name == 'United States') %>% select(5:61) %>% as.numeric
    data.frame(year = 1960:2016, val = v)
}

inflation <- extract.wb('GDP/deflator.csv') %>% mutate(price.deflator = val)
gdp <- extract.wb('GDP/gdp.csv') %>% mutate(gdp.deflator = val/last(val) * 100)
deflators <- inflation %>% mutate(gdp.deflator = gdp$gdp.deflator) %>% select(-val)
gdp.trend <- coef(lm(log(val) ~ year, data = gdp %>% filter(year > 2003)))[[2]]


In [ ]:
lookup <- dat.16 %>% 
    select(NAICS, NAICS_TITLE, TOT_EMP) %>% 
    mutate(TOT_EMP = as.numeric(TOT_EMP)) %>% 
    filter(!is.na(TOT_EMP)) %>%
    group_by(NAICS) %>% 
    summarize(employed = sum(TOT_EMP, drop.na = TRUE), NAICS_TITLE = first(NAICS_TITLE))

In [133]:
soc.xwalk <- read_xls('crosswalks/soc_2000_to_2010_crosswalk.xls', skip = 5) %>% tail(-1)

sel <- function (d, y) d %>% select(NAICS, NAICS_TITLE, OCC_CODE, OCC_TITLE, TOT_EMP, EMP_PRSE, A_MEDIAN) %>% mutate(year = y)
    
gen.occs <- function (d) d%>% filter(str_sub(OCC_CODE, -1) != '0')
    
d.03 <- dat.03 %>% 
            gen.occs() %>%
            left_join(soc.xwalk, by=c('OCC_CODE' = '2000 SOC code')) %>%
            mutate(OCC_CODE = `2010 SOC code`) %>%
            sel(2003)
    
d.07 <- sel(dat.07, 2007) %>% gen.occs()
d.12 <- sel(dat.12, 2012) %>% gen.occs()
d.16 <- sel(dat.16, 2016) %>% gen.occs()

In [230]:
gro <- d.03 %>%
    rbind(d.07, d.12, d.16) %>%
    mutate(A_MEDIAN = as.numeric(A_MEDIAN)) %>%
    filter(NAICS < 999000) %>%
    group_by(NAICS, OCC_CODE) %>%
    filter(n() == 4) %>%
    arrange(year) %>%
    mutate(growth = (A_MEDIAN - lag(A_MEDIAN))/A_MEDIAN) %>%
    ungroup() %>%
    filter(year != 2003) %>%
    mutate(TOT_EMP = as.numeric(TOT_EMP)) %>%
    filter(TOT_EMP > 5000) %>%
    filter(!is.na(growth)) %>%
    arrange(growth)

Warning message in evalq(as.numeric(A_MEDIAN), <environment>):
“NAs introduced by coercion”Warning message in evalq(as.numeric(TOT_EMP), <environment>):
“NAs introduced by coercion”

In [299]:
growths <- gro %>% 
    group_by(NAICS, OCC_CODE) %>%
    filter(n() > 2) %>%
    arrange(year) %>%
    select(growth, year, OCC_TITLE, NAICS_TITLE, A_MEDIAN) %>%
    mutate(OCC_TITLE = first(OCC_TITLE), NAICS_TITLE = first(NAICS_TITLE), OG_WAGE = first(A_MEDIAN), FINAL_WAGE = last(A_MEDIAN)) %>%
    select(-A_MEDIAN) %>%
    spread(year, growth) %>%
    ungroup()

clusts <- growths %>% select(`2007`, `2012`, `2016`) %>% kmeans(centers = 16)

Adding missing grouping variables: `NAICS`, `OCC_CODE`


In [300]:
clusts$centers %>% data.frame() %>% mutate(total_growth = X2007 + X2012 + X2016, cluster = row_number()) %>% filter(total_growth < .25)

X2007,X2012,X2016,total_growth,cluster
0.18589338,-0.11519825,0.108604772,0.1792999,2
-0.01934699,0.11411094,0.074793132,0.1695571,3
0.09767591,0.02052139,0.126046211,0.2442435,4
0.21094245,0.07826717,-0.042929298,0.2462803,6
-0.52933633,-0.01343971,0.128121143,-0.4146549,9
0.08831183,0.06873330,0.089623844,0.2466690,10
0.07124404,0.10592112,0.063137373,0.2403025,12
0.05752008,0.02946466,0.029843883,0.1168286,13
0.14220255,0.03413430,0.061622613,0.2379595,14
0.06095040,0.15307730,-0.007266758,0.2067609,16


In [302]:
growths %>% 
    mutate(cluster = clusts$cluster, total_growth = (FINAL_WAGE - OG_WAGE)/OG_WAGE) %>% 
    arrange(cluster, total_growth) %>% 
    filter(cluster == 13) %>% 
    filter(OG_WAGE < 35000)

NAICS,OCC_CODE,OCC_TITLE,NAICS_TITLE,OG_WAGE,FINAL_WAGE,2007,2012,2016,cluster,total_growth
561700,51-6011,Laundry and dry-cleaning workers,Services to Buildings and Dwellings,22080,20820,0.001358696,0.005405405,-0.066282421,13,-0.05706522
443100,49-2011,"Computer, automated teller, and office machine repairers",Electronics and Appliance Stores,31570,30250,0.092176117,-0.002222222,-0.041322314,13,-0.04181185
336300,51-9061,"Inspectors, testers, sorters, samplers, and weighers",Motor Vehicle Parts Manufacturing,34460,35350,0.027858387,0.036353468,-0.011598303,13,0.02582705
441300,41-2031,Retail salespersons,"Automotive Parts, Accessories, and Tire Stores",24040,24780,0.096505824,-0.023850085,0.052461663,13,0.03078203
442200,41-2031,Retail salespersons,Home Furnishings Stores,22880,23590,0.088286713,-0.034358047,0.062314540,13,0.03103147
441100,43-4051,Customer service representatives,Automobile Dealers,29050,29990,0.074698795,0.036803714,-0.005668556,13,0.03235800
336300,53-7051,Industrial truck and tractor operators,Motor Vehicle Parts Manufacturing,31680,33150,0.011994949,0.034440719,0.010256410,13,0.04640152
336300,51-4122,"Welding, soldering, and brazing machine setters, operators, and tenders",Motor Vehicle Parts Manufacturing,32470,34040,-0.028949800,-0.036717752,0.079905993,13,0.04835233
561300,41-2031,Retail salespersons,Employment Services,21020,22130,0.042340628,0.025498377,0.025305016,13,0.05280685
611700,25-9041,Teacher assistants,Educational Support Services,25320,26670,-0.011058452,-0.019734193,0.068991376,13,0.05331754


In [ ]:

# sel(dat.16, 2016) %>% rbind(sel(dat.03, 2003)) %>% group_by(NAICS, OCC_TITLE) %>% count()
sel(dat.16, 2016) %>% 
    rbind(sel(dat.03, 2003)) %>% 
    filter(OCC_CODE != '00-0000') %>%
    filter(NAICS == 623000) %>% 
    group_by(A_MEDIAN, EMP_PRSE, TOT_EMP) %>% 
    slice(1) %>% 
    ungroup() %>%
    arrange(OCC_CODE)

In [ ]:
mixes.16 <- get.mixes(dat.16)
mixes.03 <- get.mixes(dat.03)

Warning message in evalq(as.numeric(A_MEDIAN), <environment>):
“NAs introduced by coercion”

In [ ]:
d <- adjust(mixes.16, deflators, 2016) %>% mutate(year = 2016) %>% rbind(adjust(mixes.03, deflators, 2003) %>% mutate(year = 2003))
fin <- d %>% group_by(NAICS) %>% nest() %>% mutate(d = map(data, get.diffs)) %>% select(-data) %>% unnest()

In [95]:
joined <- fin %>% left_join(lookup, by='NAICS')
spreads <- fin %>% 
    mutate(NAICS_TITLE = joined$NAICS_TITLE, employed = joined$employed) %>% 
    arrange(spread) %>% 
    filter(employed > 50000) %>%
    mutate(lows = lows - gdp.trend*(2016-2003), highs = highs - gdp.trend*(2016-2003))

In [ ]:
prep.bls(dat.16) %>% summarize(var = var(log(A_MEDIAN))) %>% inner_join(markup.wide, by = c(NAICS = "naics4")) %>% lm(var ~ growth, data = .) %>% summary()